In [1]:
# default_exp shopping

In [2]:
import sys
sys.path

['/home/ec2-user/SageMaker/stacks/villaEcommerceBackend/backend/thinClient',
 '/home/ec2-user/SageMaker/.persisted_conda/python38/lib/python38.zip',
 '/home/ec2-user/SageMaker/.persisted_conda/python38/lib/python3.8',
 '/home/ec2-user/SageMaker/.persisted_conda/python38/lib/python3.8/lib-dynload',
 '',
 '/home/ec2-user/SageMaker/.persisted_conda/python38/lib/python3.8/site-packages',
 '/home/ec2-user/SageMaker/.persisted_conda/python38/lib/python3.8/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython']

# Shopping
simulates shopping with python code. 
this is designed to test the backend and allow conversion to frontend

In [3]:
#export
from villaProductSdk.products import ProductSdk
from villaBackendSdk.basket import BasketSdk
from awsSchema.apigateway import Response, Event
from nicHelper.wrappers import add_class_method, add_method, add_static_method
from typing import List
from s3bz.s3bz import S3
import json
from requests import post, get
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

## browsing Items

In [4]:
class Browser:
  def __init__(
    self, 
    searchEndpoint = 'https://8av9li9v82.execute-api.ap-southeast-1.amazonaws.com/Prod/orismaSearch/',
    branch = 'dev-manual'
    ):
    self.searchEndpoint = searchEndpoint
    self.productSdk = ProductSdk(branch = branch)
    

In [5]:
browser = Browser()

### Search for items

In [6]:
@add_method(Browser)
def search(self, searchQuery:str)-> List[str]:
  endpoint = self.searchEndpoint
  queries = {
    'data_t3':searchQuery
  }
  response = get(endpoint, params=queries)
  result = response.json()
  try:
    resultList = pd.DataFrame(result)['pr_code'].to_list()
  except:
    print(result)
  return resultList

In [21]:
%%time
resultList = browser.search('rose')
resultList[:10]

CPU times: user 15 ms, sys: 326 µs, total: 15.3 ms
Wall time: 1.1 s


['0193215',
 '0205723',
 '0177284',
 '0158034',
 '0214013',
 '0176563',
 '0139027',
 '0230898',
 '0175742',
 '0171289']

### get info for searched item

In [22]:
@add_method(Browser)
def getProductInfo(self,products:list)->(List[dict],List[dict]):
#   products = []
#   errors = []
  items=self.productSdk.queryList(products)
#   for result in resultList:
#     if (item := self.productSdk.(result)).get('error'):
#       products.append(item)
#     else:
#       errors.append(result)
  print(f'found {len(items)} products out of {len(products)} products')
  return items

In [23]:
%%time
products = browser.getProductInfo(resultList)
products[0]

found 225 products out of 335 products
CPU times: user 14.5 ms, sys: 4.4 ms, total: 18.9 ms
Wall time: 4.19 s


{'0193215': {'iprcode': '0193215',
  'cprcode': '0193215',
  'oprcode': '0193215',
  'ordertype': 'Y',
  'pr_abb': 'ROSE SEESENCE BONBON',
  'pr_active': 'Y',
  'pr_cgcode': '15',
  'pr_code': '0193215',
  'pr_dpcode': '19',
  'pr_engname': 'ROSE SEESENCE BONBON',
  'pr_ggcode': '195',
  'pr_market': 'บอล บอล กลิ่นกุหลาบ',
  'pr_name': 'บอล บอล กลิ่นกุหลาบ',
  'pr_puqty': '1.00',
  'pr_sa_method': '1',
  'pr_sucode1': 'CM1260',
  'pr_suref3': 'A',
  'prtype': 'I',
  'psqty': '1',
  'pstype': '1',
  'pr_country_th': '',
  'pr_country_en': 'Thailand',
  'pr_keyword_th': '',
  'pr_keyword_en': '',
  'pr_filter_th': '',
  'pr_filter_en': '',
  'online_category_l1_th': '',
  'online_category_l1_en': '',
  'online_category_l2_th': '',
  'online_category_l2_en': '',
  'online_category_l3_th': '',
  'online_category_l3_en': '',
  'villa_category_l1_en': 'Fresh',
  'villa_category_l2_en': 'Fresh Produce',
  'villa_category_l3_en': 'Bakery',
  'villa_category_l4_en': 'Cake category',
  'content_

In [ ]:
@add_method(Browser)
def getLargeImage(self, sku:str)->Image:
  url = 'http://d19oj5aeuefgv.cloudfront.net'
  return Image.open(BytesIO(requests.get(f'{url}/{sku}').content),formats=['PNG'])

In [ ]:
%%time
sku = '0189194'
browser.getLargeImage(sku)
# get image


In [ ]:
@add_method(Browser)
def getSmallImage(self, sku:str)->Image:
  url = 'https://d1vl5j0v241n75.cloudfront.net'
  return Image.open(BytesIO(requests.get(f'{url}/{sku}').content),formats=['PNG'])

In [ ]:
%%time
browser.getSmallImage(sku)
##small Image